<a href="https://colab.research.google.com/github/dmika1234/ml_uwr_22/blob/Project/Project/tfidvec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Models
from sklearn.linear_model import LogisticRegression

In [ ]:
# Importing functions
## Locally 
from funs import *

# Colab
import httpimport
with httpimport.github_repo(
    "dmika1234", "ml_uwr_22", module="Project", branch="Project"
):
    from Project.funs import *

## Loading the data

In [3]:
# Locally
# data_url = 'https://github.com/dmika1234/ml_uwr_22/blob/Project/Project/data/fake_job_postings.csv'
data_path = 'data/fake_job_postings.csv'
raw_data = pd.read_csv(data_path)

# For colab
# data_url = '/content/fake_job_postings.csv'
# raw_data = pd.read_csv(data_url, error_bad_lines=False, engine="python")
#straszny problem miałem, żeby wczytać te dane tak ja ty to robiłeś. dziwne błędy mi wyskakiwały

## Preprocessing with nltk

In [35]:
text_colnames = ['company_profile', 'description', 'requirements', 'benefits']
DataPrep = DataPreprocessor()

# text_data_ls = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=list)
text_data_np = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=np.array)
text_data_str = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=join_fun)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
working_df = raw_data.copy()

Spliting location into country, state, city

In [6]:
working_df[['country', 'state', 'city']] = working_df['location'].str.split(',', expand=True).iloc[:,0:3]

Splitting salary range into min, max salary

In [7]:
working_df[['salary_min', 'salary_max']] = working_df['salary_range'].str.split('-', expand=True)
working_df[['salary_min', 'salary_max']] = working_df[['salary_min', 'salary_max']].apply(pd.to_numeric, errors='coerce').fillna(0)

In [8]:
target_colname = 'fraudulent'
# Getting numerical colnames and deleting not useful
numerical_colnames = list(working_df.select_dtypes(include='int64').columns)
numerical_colnames = list(set(numerical_colnames) - set(['job_id', target_colname]))
numerical_colnames = numerical_colnames + ['salary_min', 'salary_max']
# Getting other text colnames and deleting not useful
other_text_colnames = list(set(working_df.select_dtypes(include='object').columns) - set(text_colnames))
other_text_colnames = list(set(other_text_colnames) - set(['location', 'salary_range']))
print(numerical_colnames)
print(text_colnames)
print(other_text_colnames)

['has_questions', 'has_company_logo', 'telecommuting', 'salary_min', 'salary_max']
['company_profile', 'description', 'requirements', 'benefits']
['required_education', 'state', 'function', 'country', 'required_experience', 'city', 'title', 'employment_type', 'industry', 'department']


Filling missing values

In [9]:
working_df[text_colnames + other_text_colnames] = working_df[text_colnames + other_text_colnames].fillna('')
working_df[numerical_colnames] = working_df[numerical_colnames].fillna(0)

In [10]:
working_df[other_text_colnames].apply(lambda x: np.unique(x).shape[0]).sort_values()

employment_type            6
required_experience        8
required_education        14
function                  38
country                   91
industry                 132
state                    326
department              1338
city                    2336
title                  11231
dtype: int64

We will only use those with not so much levels(<50 for start)

In [11]:
final_other_text_colnames = ['employment_type', 'required_experience', 'required_education', 'function']

In [12]:
X = pd.get_dummies(working_df[final_other_text_colnames], columns=final_other_text_colnames)
X[numerical_colnames] = working_df[numerical_colnames]
y = working_df[target_colname]

Splitting the data

In [13]:
train_indcs, test_indcs = get_train_test_indcs(raw_data, raw_data['fraudulent'],
 test_size=.1, random_state=42, stratify=raw_data['fraudulent'])

## BoW 

In [62]:
class BagofWords:
    def __init__(self, text_data, text_colnames) -> None:
        self.text_data = text_data
        self.text_colnames = text_colnames
        self.all_words = {}
        self.most_popular_words = {}
        self.nr_of_words = 1000
        self.onehot_dfs = {}
    def get_all_words(self):
        for column_name in self.text_colnames:
            self.all_words[column_name] = np.concatenate(self.text_data[column_name])

    def get_most_pop_words(self, nr_of_words = 1000):
        self.nr_of_words = nr_of_words
        for column_name in self.text_colnames:
            count_words = Counter(self.all_words[column_name])
            self.most_popular_words[column_name] = np.array(count_words.most_common(self.nr_of_words))[:,0]
    
    def prepare_onehot_dfs(self):
        for text_colname in self.text_colnames:
            onehot_array = np.zeros((self.text_data.shape[0], self.nr_of_words))
            for i in np.arange(self.text_data.shape[0]):
                onehot_array[i] = np.isin(self.most_popular_words[text_colname], self.text_data[text_colname][i]).astype('int64')
            column_names = np.vectorize(lambda x: text_colname + '_' + str(x))(self.most_popular_words[text_colname])
            self.onehot_dfs[text_colname] = pd.DataFrame(onehot_array, columns=column_names)

    def get_onehot_dfs(self):
        X = pd.DataFrame()
        for text_colname in self.text_colnames:
            X = pd.concat((X, self.onehot_dfs[text_colname]), axis=1)
        return X
    
    def encode_onehot(self, nr_of_words = 1000):
        self.get_all_words()
        self.get_most_pop_words(nr_of_words = 1000)
        self.prepare_onehot_dfs()
        X = self.get_onehot_dfs()
        return X

In [63]:
bow = BagofWords(text_data_np, text_colnames)
X_bow = bow.encode_onehot(1000)
X_bow_final = pd.concat((X, X_bow), axis=1)

## Using TfidVectorizer to change text to numerical vectors

In [70]:
TfTrans = TfidTranformer(50)
X_tfdif = TfTrans.vectorize_transform(text_data_str, train_indcs, text_colnames)
X_tfdif_final = pd.concat((X, X_tfdif), axis=1)

company_profile data successfuly transformed!
description data successfuly transformed!
requirements data successfuly transformed!
benefits data successfuly transformed!


# Modeling

### Train-test split

In [72]:
X_tf_train, X_tf_test, y_train, y_test = X_tfdif_final.iloc[train_indcs], X_tfdif_final.iloc[test_indcs], y[train_indcs], y[test_indcs]
X_bow_train, X_bow_test, y_train, y_test = X_bow_final.iloc[train_indcs], X_bow_final.iloc[test_indcs], y[train_indcs], y[test_indcs]

## Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier(random_state = 2137,
                                  class_weight = None)

clf_tree = clf_tree.fit(X_train, y_train)

y_pred_tree = clf_tree.predict(X_test)


In [21]:
res_tree = evaluate_performance(y_test, y_pred_tree) #None
res_tree

{'detection_percentage': 0.7471,
 'precision': 0.7471,
 'accuracy': 0.9754,
 'f1_score': 0.7471,
 'auc_roc': 0.8671}

In [22]:
res_tree = evaluate_performance(y_test, y_pred_tree) #balanced
res_tree

{'detection_percentage': 0.7471,
 'precision': 0.7471,
 'accuracy': 0.9754,
 'f1_score': 0.7471,
 'auc_roc': 0.8671}

In [23]:
[clf_tree.get_depth(), clf_tree.get_n_leaves(), clf_tree.get_params()]

[59,
 299,
 {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': None,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'random_state': 1024,
  'splitter': 'best'}]

## Cross-validation

In [24]:
from sklearn.model_selection import cross_val_score

clf_tree_cv = DecisionTreeClassifier(random_state = 2137,
                                  class_weight = None)

cross_val_score(clf_tree_cv, X, y, cv=10)

array([0.95134228, 0.96308725, 0.95917226, 0.95302013, 0.96420582,
       0.95973154, 0.95805369, 0.96308725, 0.96420582, 0.96420582])

# Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

clf_forest = RandomForestClassifier(random_state = 2137, 
                                    n_estimators = 200,
                                    class_weight = "balanced_subsample")

clf_forest = clf_forest.fit(X_train,y_train)

y_pred_forest = clf_forest.predict(X_test)

In [28]:
res_forest = evaluate_performance(y_test, y_pred_forest) #None
res_forest

{'detection_percentage': 0.5747,
 'precision': 0.9615,
 'accuracy': 0.9782,
 'f1_score': 0.7194,
 'auc_roc': 0.7868}

In [32]:
res_forest = evaluate_performance(y_test, y_pred_forest) #balanced_subsample
res_forest

{'detection_percentage': 0.5747,
 'precision': 0.9615,
 'accuracy': 0.9782,
 'f1_score': 0.7194,
 'auc_roc': 0.7868}

In [ ]:
res_forest = evaluate_performance_tree(y_test, y_pred_forest) #balanced
res_forest

{'detection_percentage': 0.6086956521739131,
 'precision': 0.9767441860465116,
 'accuracy': 0.9804332634521313,
 'f1_score': 0.75,
 'auc_roc': 0.8039807188916556}